In [1]:
import sklearn.metrics as metrics
import sklearn.linear_model as linear_model
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/austinlasseter/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
## Load the data into a DataFrame
stores=pd.read_csv('/Users/austinlasseter/DSI-EC-2/projects/datasets/stores_postEDA.csv')
stores=stores.drop(['Unnamed: 0'], axis=1)
stores.head()

,store_id,fips,metro,unemployment,income,population,ave_bottle_price,ave_number_bottles,annual_profit_per_store,number_of_stores,density,unemp_bins,income_bins,pop_bins,nstores_bins,density_bins
0,2538,19013,3.0,4.7,50887.0,131090.0,15.717369,6.556662,277246.8764,72,1820.694444,"(4, 5]","(47000, 70000]","(100000, 200000]","(60, 80]","(1000, 2000]"
1,2564,19013,3.0,4.7,50887.0,131090.0,14.465399,10.122479,186899.9286,72,1820.694444,"(4, 5]","(47000, 70000]","(100000, 200000]","(60, 80]","(1000, 2000]"
2,2575,19013,3.0,4.7,50887.0,131090.0,13.919413,7.810056,100527.7990,72,1820.694444,"(4, 5]","(47000, 70000]","(100000, 200000]","(60, 80]","(1000, 2000]"
3,2643,19013,3.0,4.7,50887.0,131090.0,17.396789,6.583914,279711.8728,72,1820.694444,"(4, 5]","(47000, 70000]","(100000, 200000]","(60, 80]","(1000, 2000]"
4,2835,19013,3.0,4.7,50887.0,131090.0,16.286067,6.998979,40534.1682,72,1820.694444,"(4, 5]","(47000, 70000]","(100000, 200000]","(60, 80]","(1000, 2000]"


In [3]:
# List out all my variables
stores.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins'],
      dtype='object')

## Model 1: All continuous variables

In [4]:
# regression model #1
dep = stores['annual_profit_per_store'] # This is the outcome I want to predict
indep = stores.drop(['store_id', 'fips',  'annual_profit_per_store',
       'number_of_stores',  'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins'], axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.142
Model:                                 OLS   Adj. R-squared:                  0.137
Method:                      Least Squares   F-statistic:                     30.22
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):           7.33e-39
Time:                             10:44:47   Log-Likelihood:                -15857.
No. Observations:                     1291   AIC:                         3.173e+04
Df Residuals:                         1283   BIC:                         3.177e+04
Df Model:                                7                                         
Covariance Type:                 nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -4.578e+04   2.66e+04     -1.720      0.086    -9.8e+04    6433.478
metro              -2269.1697    996.993     -2.276      0.023   -4225.086    -313.254
unemployment        1774.5834   2278.129      0.779      0.436   -2694.684    6243.851
income                -0.6929      0.307     -2.260      0.024      -1.295      -0.091
population             0.0111      0.015      0.759      0.448      -0.018       0.040
ave_bottle_price    7892.7384    579.829     13.612      0.000    6755.221    9030.256
ave_number_bottles   785.7100    317.604      2.474      0.013     162.631    1408.789
density                9.5690      2.388      4.007      0.000       4.885      14.253
==============================================================================
Omnibus:                      462.699   Durbin-Watson:                   1.601
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1571.679
Skew:                           1.770   Prob(JB):                         0.00
Kurtosis:                       7.085   Cond. No.                     3.47e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.47e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
# Hmm. That model only has an r2 of 14%, and 5 significant predictors. Let's try it differently

## Model 2 - Population bins

In [6]:
# Distribution of pop:
stores.pop_bins.value_counts()

(0, 30000]          487
(100000, 200000]    204
(300000, 500000]    189
(30000, 50000]      188
(70000, 100000]     117
(200000, 300000]     91
(50000, 70000]       15
Name: pop_bins, dtype: int64

In [7]:
# Let's convert population to dummy variables
model2 = pd.concat([stores, pd.get_dummies(stores['pop_bins'])], axis = 1);

In [8]:
model2.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', '(0, 30000]', '(100000, 200000]',
       '(200000, 300000]', '(30000, 50000]', '(300000, 500000]',
       '(50000, 70000]', '(70000, 100000]'],
      dtype='object')

In [9]:
# Let's see if population alone can predict store profits:

In [10]:
# regression model # 2
dep = model2['annual_profit_per_store'] # This is the outcome I want to predict
indep = model2.drop(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', '(0, 30000]',], 
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.010
Model:                                 OLS   Adj. R-squared:                  0.005
Method:                      Least Squares   F-statistic:                     2.155
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):             0.0449
Time:                             10:44:48   Log-Likelihood:                -15949.
No. Observations:                     1291   AIC:                         3.191e+04
Df Residuals:                         1284   BIC:                         3.195e+04
Df Model:                                6                                         
Covariance Type:                 nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             4.493e+04   2550.277     17.616      0.000    3.99e+04    4.99e+04
(100000, 200000]   1.23e+04   4693.656      2.620      0.009    3091.458    2.15e+04
(200000, 300000] -2938.0482   6427.329     -0.457      0.648   -1.55e+04    9671.171
(30000, 50000]    8508.4262   4832.371      1.761      0.079    -971.783     1.8e+04
(300000, 500000]  5734.2655   4823.139      1.189      0.235   -3727.832    1.52e+04
(50000, 70000]   -1.118e+04   1.48e+04     -0.758      0.449   -4.01e+04    1.78e+04
(70000, 100000]  -3991.3940   5794.458     -0.689      0.491   -1.54e+04    7376.251
==============================================================================
Omnibus:                      524.357   Durbin-Watson:                   1.448
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1803.743
Skew:                           2.043   Prob(JB):                         0.00
Kurtosis:                       7.103   Cond. No.                         9.85
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# Only one population categories(100000-200000] has a significant effect
# But the R-squared is only 1 percent

## Model 3 - Unemployment

In [12]:
# Distribution of unemployment:
stores.unemp_bins.value_counts()

(3, 4]    748
(4, 5]    286
(2, 3]    214
(5, 6]     27
(6, 7]     16
Name: unemp_bins, dtype: int64

In [13]:
# Let's convert population to dummy variables
model3 = pd.concat([stores, pd.get_dummies(stores['unemp_bins'])], axis = 1);

In [14]:
 model3.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', '(2, 3]', '(3, 4]', '(4, 5]', '(5, 6]',
       '(6, 7]'],
      dtype='object')

In [15]:
# regression model # 3
dep = model3['annual_profit_per_store'] # This is the outcome I want to predict
indep = model3.drop(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', '(2, 3]',], 
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.005
Model:                                 OLS   Adj. R-squared:                  0.001
Method:                      Least Squares   F-statistic:                     1.479
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):              0.206
Time:                             10:44:48   Log-Likelihood:                -15953.
No. Observations:                     1291   AIC:                         3.192e+04
Df Residuals:                         1286   BIC:                         3.194e+04
Df Model:                                4                                         
Covariance Type:                 nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.186e+04   3854.657     13.455      0.000    4.43e+04    5.94e+04
(3, 4]     -6135.4284   4371.419     -1.404      0.161   -1.47e+04    2440.467
(4, 5]     -2087.1208   5096.684     -0.410      0.682   -1.21e+04    7911.606
(5, 6]      1.201e+04   1.15e+04      1.043      0.297   -1.06e+04    3.46e+04
(6, 7]       1.22e+04   1.46e+04      0.835      0.404   -1.65e+04    4.09e+04
==============================================================================
Omnibus:                      526.328   Durbin-Watson:                   1.436
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1818.064
Skew:                           2.050   Prob(JB):                         0.00
Kurtosis:                       7.122   Cond. No.                         11.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# None of the unemployment brackets have a sig effect on annual profits
# The model has an R2 of only 5%
# Let's drop unemployment from our final model

## Model 3 - Income

In [17]:
# Distribution of county-level median household income:
stores.income_bins.value_counts()

(47000, 70000]     1219
(70000, 100000]      37
(40000, 47000]       24
(0, 40000]           11
Name: income_bins, dtype: int64

In [18]:
# Let's convert to dummy variables
model4 = pd.concat([stores, pd.get_dummies(stores['income_bins'])], axis = 1);

In [19]:
model4.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', '(0, 40000]', '(40000, 47000]',
       '(47000, 70000]', '(70000, 100000]'],
      dtype='object')

In [20]:
# regression model # 4
dep = model4['annual_profit_per_store'] # This is the outcome I want to predict
indep = model4.drop(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', '(0, 40000]'],
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.004
Model:                                 OLS   Adj. R-squared:                  0.001
Method:                      Least Squares   F-statistic:                     1.607
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):              0.186
Time:                             10:44:48   Log-Likelihood:                -15953.
No. Observations:                     1291   AIC:                         3.191e+04
Df Residuals:                         1287   BIC:                         3.194e+04
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            3.634e+04    1.7e+04      2.137      0.033    2986.937    6.97e+04
(40000, 47000]   2.924e+04   2.05e+04      1.424      0.155    -1.1e+04    6.95e+04
(47000, 70000]   1.208e+04   1.71e+04      0.707      0.480   -2.14e+04    4.56e+04
(70000, 100000] -1370.3603   1.94e+04     -0.071      0.944   -3.94e+04    3.66e+04
==============================================================================
Omnibus:                      523.555   Durbin-Watson:                   1.438
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1794.079
Skew:                           2.042   Prob(JB):                         0.00
Kurtosis:                       7.084   Cond. No.                         30.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
# None of the income brackets has a sig effect on annual profits
# The model has an R2 of only 4%

## Model 5 - Rural Bins

In [22]:
# What happens if we convert metro/rural to a dummy?
model5 = pd.concat([stores, pd.get_dummies(stores['metro'])], axis = 1);

In [23]:
model5.columns

Index([               'store_id',                    'fips',
                         'metro',            'unemployment',
                        'income',              'population',
              'ave_bottle_price',      'ave_number_bottles',
       'annual_profit_per_store',        'number_of_stores',
                       'density',              'unemp_bins',
                   'income_bins',                'pop_bins',
                  'nstores_bins',            'density_bins',
                             2.0,                       3.0,
                             4.0,                       5.0,
                             6.0,                       7.0,
                             8.0,                       9.0],
      dtype='object')

In [24]:
# regression model # 5a
dep = model5['annual_profit_per_store'] # This is the outcome I want to predict
indep = model5.drop(['store_id',                    'fips',
                         'metro',            'unemployment',
                        'income',              'population',
              'ave_bottle_price',      'ave_number_bottles',
       'annual_profit_per_store',        'number_of_stores',
                       'density',              'unemp_bins',
                   'income_bins',                'pop_bins',
                  'nstores_bins',            'density_bins',
                             2.0],
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.021
Model:                                 OLS   Adj. R-squared:                  0.016
Method:                      Least Squares   F-statistic:                     4.002
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):           0.000243
Time:                             10:44:48   Log-Likelihood:                -15942.
No. Observations:                     1291   AIC:                         3.190e+04
Df Residuals:                         1283   BIC:                         3.194e+04
Df Model:                                7                                         
Covariance Type:                 nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.654e+04   2630.009     17.697      0.000    4.14e+04    5.17e+04
3.0         6551.5305   4433.312      1.478      0.140   -2145.806    1.52e+04
4.0        -4532.2736   8418.042     -0.538      0.590    -2.1e+04     1.2e+04
5.0         1.754e+04   6521.005      2.689      0.007    4745.166    3.03e+04
6.0         4326.5066   4820.438      0.898      0.370   -5130.300    1.38e+04
7.0         3186.9394   5034.737      0.633      0.527   -6690.282    1.31e+04
8.0        -2.163e+04   8195.987     -2.639      0.008   -3.77e+04   -5547.691
9.0        -2.382e+04   9129.118     -2.609      0.009   -4.17e+04   -5910.264
==============================================================================
Omnibus:                      507.854   Durbin-Watson:                   1.458
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1686.526
Skew:                           1.987   Prob(JB):                         0.00
Kurtosis:                       6.944   Cond. No.                         6.81
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
# A couple of metro values (5, 8, and 9) have a sig effect on the outcome, but in opposite directions.
# Let's try a different approach
# Suburbs
stores['Bool_Suburban']=stores['metro']==5
print(stores['Bool_Suburban'].value_counts())
stores = pd.concat([stores, pd.get_dummies(stores['Bool_Suburban'])], axis = 1);
stores.drop(['Bool_Suburban', False], axis=1, inplace=True)
stores.rename(columns = {True: 'suburbs'}, inplace=True)
stores.columns

False    1203
True       88
Name: Bool_Suburban, dtype: int64


Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs'],
      dtype='object')

In [26]:
# Towns
stores['Bool_Town']=(stores['metro']==6) | (stores['metro']==7)
print(stores['Bool_Town'].value_counts())
stores = pd.concat([stores, pd.get_dummies(stores['Bool_Town'])], axis = 1);
stores.drop(['Bool_Town', False], axis=1, inplace=True)
stores.rename(columns = {True: 'town'}, inplace=True)
stores.columns

False    929
True     362
Name: Bool_Town, dtype: int64


Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town'],
      dtype='object')

In [27]:
# Rural
stores['Bool_Rural']=(stores['metro']>7)
print(stores['Bool_Rural'].value_counts())
stores = pd.concat([stores, pd.get_dummies(stores['Bool_Rural'])], axis = 1);
stores.drop(['Bool_Rural', False], axis=1, inplace=True)
stores.rename(columns = {True: 'rural'}, inplace=True)
stores.columns

False    1198
True       93
Name: Bool_Rural, dtype: int64


Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural'],
      dtype='object')

In [28]:
stores.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural'],
      dtype='object')

In [29]:
# Let's run a new regression model with these dummies
# regression model # 5b
dep = stores['annual_profit_per_store'] # This is the outcome I want to predict
indep = stores.drop(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', ],  axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.019
Model:                                 OLS   Adj. R-squared:                  0.017
Method:                      Least Squares   F-statistic:                     8.365
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):           1.65e-05
Time:                             10:44:48   Log-Likelihood:                -15943.
No. Observations:                     1291   AIC:                         3.189e+04
Df Residuals:                         1287   BIC:                         3.192e+04
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        4.84e+04   2045.863     23.657      0.000    4.44e+04    5.24e+04
suburbs     1.568e+04   6305.774      2.487      0.013    3309.699    2.81e+04
town        1933.6033   3582.480      0.540      0.589   -5094.538    8961.745
rural      -2.445e+04   6152.238     -3.974      0.000   -3.65e+04   -1.24e+04
==============================================================================
Omnibus:                      511.848   Durbin-Watson:                   1.455
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1720.858
Skew:                           1.998   Prob(JB):                         0.00
Kurtosis:                       7.002   Cond. No.                         4.54
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# Okay, this seems promising. 
# We see a significant effect for suburban (positive) and rural (negative) compared to urban.

## Model 6: Density

In [31]:
# What happens if we convert metro/rural to a dummy?
model6 = pd.concat([stores, pd.get_dummies(stores['density_bins'])], axis = 1);

In [32]:
model6.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural', '(0, 1000]',
       '(1000, 2000]', '(2000, 3000]', '(3000, 4000]', '(4000, 10000]'],
      dtype='object')

In [33]:
# regression model # 7
dep = model6['annual_profit_per_store'] # This is the outcome I want to predict
indep = model6.drop(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural', '(0, 1000]',
       '(1000, 2000]', '(2000, 3000]'], 
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.010
Model:                                 OLS   Adj. R-squared:                  0.008
Method:                      Least Squares   F-statistic:                     6.268
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):            0.00195
Time:                             10:44:48   Log-Likelihood:                -15950.
No. Observations:                     1291   AIC:                         3.191e+04
Df Residuals:                         1288   BIC:                         3.192e+04
Df Model:                                2                                         
Covariance Type:                 nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          4.666e+04   1636.785     28.504      0.000    4.34e+04    4.99e+04
(3000, 4000]   1.441e+04   6382.376      2.257      0.024    1884.357    2.69e+04
(4000, 10000]  2.974e+04   1.06e+04      2.815      0.005    9017.775    5.05e+04
==============================================================================
Omnibus:                      531.598   Durbin-Watson:                   1.445
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1872.150
Skew:                           2.063   Prob(JB):                         0.00
Kurtosis:                       7.218   Cond. No.                         6.78
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Model 7 - Final Model

In [34]:
stores.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural'],
      dtype='object')

#### Model 7a: income, density, and the metro dummies

In [35]:
# regression model # 7a
dep = stores['annual_profit_per_store'] # This is the outcome I want to predict
indep = stores.drop(['store_id', 'fips', 'metro', 'unemployment',  'population',
        'annual_profit_per_store',
       'number_of_stores',  'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins'], 
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.150
Model:                                 OLS   Adj. R-squared:                  0.145
Method:                      Least Squares   F-statistic:                     32.37
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):           1.36e-41
Time:                             10:44:48   Log-Likelihood:                -15851.
No. Observations:                     1291   AIC:                         3.172e+04
Df Residuals:                         1283   BIC:                         3.176e+04
Df Model:                                7                                         
Covariance Type:                 nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -5.97e+04   1.77e+04     -3.364      0.001   -9.45e+04   -2.49e+04
income                -0.3593      0.250     -1.439      0.150      -0.849       0.131
ave_bottle_price    7786.1751    574.078     13.563      0.000    6659.941    8912.409
ave_number_bottles   756.2020    315.638      2.396      0.017     136.979    1375.425
density                8.2945      2.311      3.590      0.000       3.762      12.827
suburbs             1.212e+04   6541.873      1.853      0.064    -712.449     2.5e+04
town               -3716.8030   3760.933     -0.988      0.323   -1.11e+04    3661.450
rural              -2.611e+04   6162.575     -4.236      0.000   -3.82e+04    -1.4e+04
==============================================================================
Omnibus:                      449.191   Durbin-Watson:                   1.609
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1478.241
Skew:                           1.727   Prob(JB):                         0.00
Kurtosis:                       6.943   Cond. No.                     7.19e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.19e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Model 7b. Density, Income dummies, and Metro

In [36]:
# Dummify income.
model7b = pd.concat([stores, pd.get_dummies(stores['income_bins'])], axis = 1);

In [37]:
model7b.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural',
       '(0, 40000]', '(40000, 47000]', '(47000, 70000]', '(70000, 100000]'],
      dtype='object')

In [38]:
# regression model # 7b
dep = model7b['annual_profit_per_store'] # This is the outcome I want to predict
indep = model7b.drop(['store_id', 'fips', 'unemployment', 'income', 'population',
       'annual_profit_per_store',
       'number_of_stores', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural',
       '(0, 40000]'], 
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.141
Model:                                 OLS   Adj. R-squared:                  0.136
Method:                      Least Squares   F-statistic:                     30.06
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):           1.20e-38
Time:                             10:44:48   Log-Likelihood:                -15858.
No. Observations:                     1291   AIC:                         3.173e+04
Df Residuals:                         1283   BIC:                         3.177e+04
Df Model:                                7                                         
Covariance Type:                 nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -8.806e+04   1.97e+04     -4.481      0.000   -1.27e+05   -4.95e+04
metro              -1826.9938    700.377     -2.609      0.009   -3201.004    -452.983
ave_bottle_price    7843.0057    577.123     13.590      0.000    6710.798    8975.214
ave_number_bottles   878.8674    316.071      2.781      0.006     258.795    1498.940
density                9.1529      2.385      3.837      0.000       4.474      13.832
(40000, 47000]       2.35e+04   1.93e+04      1.220      0.223   -1.43e+04    6.13e+04
(47000, 70000]      1.012e+04   1.61e+04      0.629      0.530   -2.15e+04    4.17e+04
(70000, 100000]    -1.434e+04   1.86e+04     -0.771      0.441   -5.08e+04    2.22e+04
==============================================================================
Omnibus:                      461.245   Durbin-Watson:                   1.602
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1554.823
Skew:                           1.768   Prob(JB):                         0.00
Kurtosis:                       7.051   Cond. No.                     5.27e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
# Median Household Income just doesn't have a significant impact once you include metro status.
# We'll have to drop it from the model.

#### Model 7c: Drop income; keep density and the metro dummies

In [44]:
# regression model # 7c
dep = stores['annual_profit_per_store'] # This is the outcome I want to predict
indep = stores.drop(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'annual_profit_per_store',
       'number_of_stores', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins'],
                    axis = 'columns') # These are the features that predict it
indep = sm.add_constant(indep) # Add the intercept
model = sm.OLS(dep,indep) # Instantiate the model
results = model.fit() # Fit the model
results.summary() # Summarize the results

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     annual_profit_per_store   R-squared:                       0.149
Model:                                 OLS   Adj. R-squared:                  0.145
Method:                      Least Squares   F-statistic:                     37.39
Date:                     Thu, 08 Feb 2018   Prob (F-statistic):           5.83e-42
Time:                             10:46:55   Log-Likelihood:                -15852.
No. Observations:                     1291   AIC:                         3.172e+04
Df Residuals:                         1284   BIC:                         3.175e+04
Df Model:                                6                                         
Covariance Type:                 nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -8.045e+04   1.03e+04     -7.773      0.000   -1.01e+05   -6.01e+04
ave_bottle_price    7775.1602    574.266     13.539      0.000    6648.558    8901.763
ave_number_bottles   781.6108    315.275      2.479      0.013     163.101    1400.121
density                7.8649      2.292      3.431      0.001       3.368      12.362
suburbs             1.621e+04   5895.357      2.749      0.006    4642.786    2.78e+04
town               -1332.6517   3377.547     -0.395      0.693   -7958.768    5293.465
rural              -2.338e+04   5865.936     -3.986      0.000   -3.49e+04   -1.19e+04
==============================================================================
Omnibus:                      449.601   Durbin-Watson:                   1.607
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1479.830
Skew:                           1.729   Prob(JB):                         0.00
Kurtosis:                       6.944   Cond. No.                     1.66e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.66e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Let's go with Model 7c: It has 5 sig predictors and an R-sq of 15%.

In [45]:
# Rural stores earn $23K less on average than urban stores.
# Suburban stores earn $16K more than urban stores.
# For a 100-person increase in the density of people per store, annual profit increases by $786
# For a one-bottle increase in the average # of bottles per transaction, annual profits increase by $780.
# For a one-dollar increase in the average $ of a bottle in a transaction, annual profits increase by almost $8K.
# The R-squared is 15% -- low, but respectable.

In [42]:
# Save your work to a csv
stores.to_csv('/Users/austinlasseter/DSI-EC-2/projects/datasets/stores_modeling.csv')